In [ ]:
import numpy
import scipy
print(f"NumPy file: {numpy.__file__}")
print(f"NumPy version: {numpy.__version__}")
print(f"SciPy file: {scipy.__file__}")
print(f"SciPy version: {scipy.__version__}")

In [ ]:
!pip install distilabel[openai] openai pandas

In [ ]:
import sys
import importlib

def check_version(package_name):
    try:
        lib = importlib.import_module(package_name)
        print(f"✅ {package_name:<15} : {lib.__version__}")
    except ImportError:
        print(f"❌ {package_name:<15} : 未安装")
    except Exception as e:
        print(f"⚠️ {package_name:<15} : 导入出错 ({e})")

print(f"Python 版本: {sys.version.split()[0]}")
print("-" * 30)

# 检查核心依赖
pkgs = ['vllm', 'numpy', 'scipy', 'transformers', 'torch']
for pkg in pkgs:
    check_version(pkg)

In [ ]:
# 1. 验证导入是否成功
import distilabel
from distilabel.llms import OpenAILLM
from distilabel.pipeline import Pipeline

print(f"Distilabel 版本: {distilabel.__version__}")
print("环境验证成功，可以开始连接 vLLM 了！")

In [ ]:
!env | grep -i SERVICE

In [ ]:
import subprocess
import os

# 1. 刚才下载的绝对路径
MODEL_PATH = "/data/models/Qwen2.5-7B-Instruct"

# 2. 启动 vLLM
command = [
    "python3", "-m", "vllm.entrypoints.openai.api_server",
    "--model", MODEL_PATH,            # 指向本地路径
    "--served-model-name", "Qwen",    # 给模型起个简单的名字
    "--trust-remote-code",
    "--max-model-len", "8192",
    "--gpu-memory-utilization", "0.95",
    "--port", "8000"
]

print(f"🚀 正在加载本地模型: {MODEL_PATH}")

with open("vllm.log", "w") as log_file:
    process = subprocess.Popen(
        command,
        env=os.environ, # 继承当前环境变量
        stdout=log_file,
        stderr=log_file,
        text=True
    )

print(f"✅ 后台进程 PID: {process.pid}")
print("👀 请运行 '!tail -f vllm.log' 查看加载进度，直到看见 'Uvicorn running'")

In [ ]:
import subprocess
import os

# 1. 刚才下载的绝对路径
MODEL_PATH = "/data/models/Qwen3-8B"

# 2. 启动 vLLM
command = [
    "python3", "-m", "vllm.entrypoints.openai.api_server",
    "--model", MODEL_PATH,            # 指向本地路径
    "--served-model-name", "Qwen",    # 给模型起个简单的名字
    "--trust-remote-code",
    "--max-model-len", "8192",
    "--gpu-memory-utilization", "0.95",
    "--port", "8000"
]

print(f"🚀 正在加载本地模型: {MODEL_PATH}")

with open("vllm.log", "w") as log_file:
    process = subprocess.Popen(
        command,
        env=os.environ, # 继承当前环境变量
        stdout=log_file,
        stderr=log_file,
        text=True
    )

print(f"✅ 后台进程 PID: {process.pid}")
print("👀 请运行 '!tail -f vllm.log' 查看加载进度，直到看见 'Uvicorn running'")

In [ ]:
import os
import sys
import requests
import pandas as pd
from distilabel.models import OpenAILLM
from distilabel.pipeline import Pipeline
from distilabel.steps.tasks import TextGeneration
from distilabel.steps import LoadDataFromDicts

# ================= ⚙️ 1. 基础环境配置 =================
# 强制不走代理，防止 localhost 连接失败
os.environ["no_proxy"] = "localhost,127.0.0.1,0.0.0.0"
os.environ["NO_PROXY"] = "localhost,127.0.0.1,0.0.0.0"

# vLLM 服务地址
BASE_URL = "http://localhost:8000/v1" 

print("🕵️‍♂️ 正在检测 vLLM 服务状态...")
try:
    # 自动获取真实模型名称，防止 404 错误
    resp = requests.get(f"{BASE_URL}/models", timeout=3)
    if resp.status_code == 200:
        # 获取列表中的第一个模型ID
        MODEL_NAME = resp.json()['data'][0]['id']
        print(f"✅ 服务已连接！锁定模型 ID: [{MODEL_NAME}]")
    else:
        raise Exception(f"服务响应错误: {resp.text}")
except Exception as e:
    print(f"❌ 无法连接 vLLM (请确认服务端口 8000 已开启): {e}")
    sys.exit(1)

# ================= 🧠 2. 定义【深度重构】提示词 =================
# 核心策略：先拆解信息，后重组，禁止直接翻译，要求风格剧烈变化
DEEP_REWRITE_SYSTEM_PROMPT = """你是一名资深的反抄袭专家和教科书作家。
你的任务是将用户提供的文本进行【彻底重构】，使其在查重软件中检测率为 0%，同时保留原文核心信息。

请严格按照以下【思维链】在内心进行处理（不要输出思考过程）：
1. 【提取】：提取原文的 3-5 个核心事实点。
2. 【粉碎】：彻底遗忘原文的人名（如"波特夫妇"）、句式结构、连接词（如"因为...所以"）和叙述顺序。
3. 【重塑】：基于提取的事实点，用全新的逻辑和口吻重新撰写。

⚠️ 严格禁忌：
1. 严禁连续 5 个字与原文相同。
2. 严禁保留原文的人名。
3. 严禁保留原文的逻辑连接词，必须使用完全不同的表达方式。
4. 必须打乱原本的信息叙述顺序（例如：原文先说背景后说结论，你就要先抛出结论再补充背景）。


请直接输出重写后的最终文本。
"""

# ================= 🧪 3. 准备待改写数据 =================
raw_texts = [
    {
        "original_text": "午休时间，当弗农到楼下马路对角的面包房中购买午餐时，他再次见到一群穿着奇装异服的人。尽管这些人都在悄声说话，但弗农还是听到了只言片语：波特夫妇和他们的儿子哈利。弗农感到万分惊恐，因为波特是佩妮的妹夫。如果人们发现他们和波特家有关系，那么弗农和佩妮一定会非常尴尬，因为波特夫妇在某些方面很怪异。弗农下班时，他和一个小老头撞了个满怀。他跟弗农说，神秘人已经被打败了，就连像弗农这样的麻瓜也应该庆祝。弗农并不知道麻瓜是什么，感觉受到了冒犯。"
    },
    {
        "original_text": "在Python中，列表推导式提供了一种简洁的方法来创建列表。常见的用法是针对序列中的每个元素执行某些操作，或者创建一个满足特定条件的子序列。"
    },
    {
        "original_text": "由于不可抗力因素，本次航班将被迫取消。航空公司将为受影响的旅客提供全额退款或免费改签服务，详情请咨询柜台工作人员。"
    }
]

# ================= ⛓️ 4. 构建流水线 =================
print("🚀 正在构建深度重构流水线...")

with Pipeline(name="deep-rewrite-pipeline") as pipeline:
    
    # 加载数据
    loader = LoadDataFromDicts(data=raw_texts)

    # 改写生成器
    rewriter = TextGeneration(
        name="deep_rewriter",
        llm=OpenAILLM(
            base_url=BASE_URL,
            model=MODEL_NAME,
            api_key="EMPTY",
            timeout=120,
        ),
        system_prompt=DEEP_REWRITE_SYSTEM_PROMPT,
        
        # ⚠️ 关键映射：将数据里的 'original_text' 列作为 'instruction' 输入
        input_mappings={"instruction": "original_text"},
        
        # 💡 强力模板：要求使用“知乎/博客风格”，强制打破原文语体
        template="""原文内容：
{{ instruction }}

请假设你正在打谜语，用"只提供线索，不给具体细节"的风格，向读者重新解释上述内容。
确保看起来完全不像原来的句子：""", 
        
        # 输出重命名
        output_mappings={"generation": "rewritten_text"}, 
    )

    # 连接步骤
    loader >> rewriter

# ================= 🏃 5. 运行与展示 =================
if __name__ == "__main__":
    print(f"⏳ 开始深度改写 {len(raw_texts)} 条文本...")
    print("   (Temperature 设置为 0.85 以增加差异度)")
    
    try:
        distiset = pipeline.run(
            parameters={
                rewriter.name: {
                    "llm": {
                        "generation_kwargs": {
                            # ⚠️ 高温参数：0.85 能让模型发挥更多创造力，避免死板翻译
                            "temperature": 0.85, 
                            "top_p": 0.95,
                            "max_new_tokens": 1024,
                        }
                    }
                }
            },
            use_cache=False
        )

        # ================= 📊 结果对比 =================
        df = distiset["default"]["train"].to_pandas()
        
        print("\n" + "="*60)
        print("📝 改写效果对比 (Original vs Rewritten)：")
        print("="*60)
        
        for index, row in df.iterrows():
            print(f"🔴 [原文]: {row['original_text']}")
            print(f"🟢 [重写]: {row['rewritten_text']}")
            print("-" * 60)
        
        # 保存结果
        output_file = "deep_rewritten_results.json"
        df.to_json(output_file, force_ascii=False, orient="records", indent=2)
        print(f"\n💾 结果已保存至: {os.path.abspath(output_file)}")

    except Exception as e:
        print(f"\n❌ 流水线运行出错: {e}")
        import traceback
        traceback.print_exc()